In [1]:
import os
import json
import pandas as pd

In [2]:
file_path = '../NewsEdits/data/newsedits_edits_pairs.csv'
# Step 1: Verify the file path
if not os.path.exists(file_path):
    print("File not found:", file_path)
else:
    print("File found:", file_path)

df = pd.read_csv(file_path)

# Drop a column (e.g., 'column2')
df = df.drop('Unnamed: 0', axis=1)
df

File found: ../NewsEdits/data/newsedits_edits_pairs.csv


,before,after
0,< p > Lopes was arrested and rushed to a hospi...,< p > Lopes was arrested and taken to a hospit...
1,"< p > According to the letter , a uniformed Bo...",The mayor and his security detail were spotte...
2,"< p > According to the letter , a uniformed Bo...","< p > According to the letter , a uniformed Bo..."
3,"< p > According to the letter , a uniformed Bo...","< p > According to the letter , a uniformed Bo..."
4,< p > Undated file photo shows Emmett Louis Ti...,< /p > < p > Undated file photo shows Emmett ...
...,...,...
15228,( Palm Beach County Sheriff 's Office ) < /p >...,< p > Newly released video shows the moment a...
15229,< /p > < p > While the winning team from Brita...,< p > While the winning team from Britain cro...
15230,"< /p > < p > While holding his baby daughter ,...","< p > While holding his baby daughter , James..."
15231,"< /p > < p > A team of experts from Israel , t...","< p > A team of experts from Israel , the U.S..."


In [3]:
repo="NewsEdits"
n_sampels_pilot_study=100
n_sampels_pilot_study_list=[30, 35, 35]
n_samples_comparing_group=1000
random_seed=41

# repo="arXivEdits"
# n_sampels_pilot_study=15
# n_sampels_pilot_study_list=[5, 5, 5]
# n_samples_comparing_group=165
# random_seed=41
# group_by=['label']

In [4]:
# Define the sampling function
def sample_data(group, n=1, random_state=42):
    return group.sample(n=min(len(group), n), random_state=random_state)

# Sample n rows per 'domain' and 'label' combination with a random seed
# sampled_df = df.groupby(group_by).apply(sample_data, n=n_sampels_pilot_study, random_state=random_seed).reset_index(drop=True)
sampled_df = df.sample(n=n_sampels_pilot_study, random_state=random_seed)

# Display the sampled DataFrame
sampled_df

,before,after
3260,< /p > < p > `` Donald Trump has done a lot of...,`` Donald Trump has done a lot of horrible th...
4594,< /p > < p > Flake was unsupportive of Preside...,< /p > < p > Flake was unsupportive of Preside...
13410,( Truman Little White House ) < /p > < p > '' ...,< /p > < p > '' While they are higher-end fur...
6073,This is on Publish with no configured Image < ...,< p > Chick-fil-A claims to have given away a...
13858,< p > < /p > < p > A federal judge dismissed a...,< p > < /p > < p > Taylor Swift testified Thur...
...,...,...
3071,"< p > Earlier Saturday , Hawaiian citizens rep...","< p > At about 8:07 a.m. local time , Hawaiian..."
12492,"< /p > < p > According to Stars and Stripes , ...","< p > According to Stars and Stripes , the U...."
13407,< /p > < p > Click for more from Fox 7. < /p >...,< /p > < p > Click for more from Fox 7. < /p >
1185,"Jason Chaffetz , R-Utah , in 2015. < /p > < p ...",< p > Rhee is a former partner at WilmerHale ...


In [5]:
# Get the indices of the sampled data
sampled_indices = sampled_df.index

# Drop the sampled data to get the remaining data
remaining_df = df.drop(sampled_indices)

remaining_df = remaining_df.reset_index(drop=True)

In [6]:
# Convert DataFrame to JSON Lines format and save to a file
sampled_df.to_json(f"./{repo}/pilot_study_{repo.lower()}.json", orient='records', lines=True)

In [7]:

for idx, n_samples in enumerate(n_sampels_pilot_study_list):
    # Sample n rows per 'domain' and 'label' combination with a random seed
    # sampled_iter_df = sampled_df.groupby(group_by).apply(sample_data, n=n_samples, random_state=random_seed).reset_index(drop=True)
    sampled_iter_df = sampled_df.sample(n=n_samples, random_state=random_seed)

    sampled_iter_df.to_json(f"./{repo}/pilot_study_{repo.lower()}_iter{idx+1}.json", orient='records', lines=True)

    # Get the indices of the sampled data
    sampled_iter_indices = sampled_iter_df.index

    # Drop the sampled data to get the remaining data
    sampled_df = sampled_df.drop(sampled_iter_indices)
    # Reset the index
    sampled_df = sampled_df.reset_index(drop=True)

In [8]:
## dataset for comparing group

# Sample n rows per 'domain' and 'label' combination with a random seed
# comparing_df = remaining_df.groupby(group_by).apply(sample_data, n=n_samples_comparing_group, random_state=random_seed).reset_index(drop=True)
comparing_df = remaining_df.sample(n=n_samples_comparing_group, random_state=random_seed).reset_index(drop=True)

comparing_df.to_json(f"./{repo}/comparing_group_{repo.lower()}.json", orient='records', lines=True)